In [156]:
import requests as req
from bs4 import BeautifulSoup
import json
import xlrd

# Get HTML Content

In [55]:
response=req.get('https://www.mygreatlearning.com/academy/')
soup=BeautifulSoup(response.text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--suppress HtmlRequiredTitleElement -->
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <!--  <link rel="manifest" href="/manifest.json">-->
  <meta content="#026adb" name="theme-color">
   <title>
    Best Free Online Courses with Certificates [2023] - Great Learning
   </title>
   <meta content="Great Learning Academy offers free online certificate courses in various domains such as Data Science, AI, ML, IT &amp; Software, Cloud Computing, Marketing, Big Data &amp; more." name="description"/>
   <meta content="great learning academy, great learning free courses, free online courses, free online courses with certificates, free online classes, best online courses, best free free online courses, best free online courses with certificates, short courses online, gl academy" name="keywords"/>
   <link href="https://www.mygreatlearning.com/academy" rel="canonical"/>
   <meta content="en_US" property="og:locale"/>
   <meta content="website" property="og:t

# Creating Courses Dictionary

In [115]:
courses={"name":[],"duration":[],"learners":[],"level":[],"rating":[],"url":[]}

# Get Courses Name

In [119]:
for course in soup.select('.new_course_card_v2_details_section h3'):
    courses['name'].append(course.get_text())

# Get Ratings

In [120]:
for rating in soup.select('.star-ratings-container-sm span'):
    rate=rating.get_text()
    if rate!='':
        courses['rating'].append(rate)

# Get learners & level

In [121]:
for span in soup.select('.new_course_dot'):
    text=span.get_text()
    if text in ['Beginner','Intermediate']:
        courses['level'].append(text)
    else:
        courses['learners'].append(text)

# Course Duration

In [122]:
for duration in soup.select('.academy-course-content-section span'):
    courses['duration'].append(duration.get_text())

# Course Link

In [123]:
for link in soup.select('.academy_v2_course_container a'):
    link=link.get('href')
    if link.startswith('https'):
        courses['url'].append(link)

# Export data into Excel

In [160]:
courses_df=pd.DataFrame(courses)
courses_df.to_excel('courses.xlsx',index=False)

# Export Courses in JSON File

In [222]:
courses=[]
for course in soup.find_all('div',class_='academy_v2_course_container'):
    name=course.find('h3').get_text()
    rating=course.select('.star-ratings-container-sm span')[-1].get_text()
    link=course.find('a').get('href')
    level_learners=course.find_all('span',class_='new_course_dot')
    learners,level=level_learners[0].get_text(),level_learners[1].get_text()
    span=course.find('div',class_='academy-course-content-section')
    duration=list(span.children)[3].text
    courses.append({'name':name,'duration':duration,'ratings':rating,'learners':learners,'levels':level,'course_link':link})
    
#courses
with open('courses.json','w') as file:
    json.dump(courses,file)